# 使用LSTM生成文本

### Step 1：准备数据

In [1]:
# 下载并解析初始文本文件

import keras 
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600893


In [2]:
# 将字符序列向量化 
maxlen = 60

step = 3

sentences = []

next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Number of sequences:', len(sentences))

Number of sequences: 200278


In [3]:
chars = sorted(list(set(text)))
chars
print('Unique characters:', len(chars))

Unique characters: 58


In [4]:
char_indices = dict((char, chars.index(char)) for char in chars) 
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool) 
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1
print('End...')

Vectorization...


C:\Users\HP\AppData\Local\Temp\ipykernel_10264\670610677.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
C:\Users\HP\AppData\Local\Temp\ipykernel_10264\670610677.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


End...


In [5]:
print(len(x[0]))
print(len(x[0][0]))

60
58


### Step 2：构建网络

In [6]:
from keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
          
# 模型编译配置
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

### Step 3：训练语言模型并从中采样

In [7]:
# 给定模型预测，采样下一个字符的函数
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

import random
import sys

for epoch in range(1, 60):
    print('---- epoch: ', epoch, ' ----\n')
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text)- maxlen - 1)
    # 随机选择一个文本种子
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + ' --- " \n')
    for temperature in [0.2, 0.5, 1.0, 1.5]:
        print('------ temperature:', temperature, ' ------\n')
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            # 对目前生成的字符进行one-hot 编码
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
                
            preds = model.predict(sampled, verbose=0)[0] 
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)

---- epoch:  1  ----

1565/1565 [==============================] - 124s 78ms/step - loss: 1.9623
--- Generating with seed: "od might be mixed with my evils (illness, loneliness,
strang --- " 

------ temperature: 0.2  ------

od might be mixed with my evils (illness, loneliness,
strange and the man of the moder the strenger to the man it is a soultine of the ence to belief the will and the man and the prome and the man with have the prome the sention of the self-condicial and the more and the relight and the moral and consideres and such a self-consided the accouse the moder and the will and the moderation of the great the man and the self-consident and the cause the free the s------ temperature: 0.5  ------

 the man and the self-consident and the cause the free the stult of the ederation of the persing of the conder. the canvally the will chomsess, and stacding in the condention, of a men of the most and human to seed at accome to the man love as the great as account and decernt it. 

ly the mouals from accordinance, and the wild would have to conspudatate what wowars atacc,--horedhally-scwendljoucbly dypwsper. onatarlys., cestarazs that blpee: i knows, or i cally thews thepe thephowy upon 2fly-
gutiti?s of pucity:  then, que desirem, wellatized,
irled? mutt iting ispodage, varialnes, ed.. but a syattefilting,"hony.ww
viatiois ow rul-word usfinious he
woms ansile-talitnwnis.ply most dam. includingatolint of the
ospeedificaking eserumoun---- epoch:  5  ----

1565/1565 [==============================] - 133s 85ms/step - loss: 1.4506
--- Generating with seed: "g things: he then
perceives, perhaps, that those effects cou --- " 

------ temperature: 0.2  ------

g things: he then
perceives, perhaps, that those effects course and sorious and discasion and the other in the world and the preference of the spirit and the spirit in the stronger and the individual and the sufficiently and and strength and in the other and the stronger the antaments of the experiences and does 

e spirits: the strength of the struggle of the  recognized boy four from him will oberest respendle to ruces, i tempose
man iself, he on at, all it is nonce and
stupiden faith to modern manest
lawadines possence, then, and who seret to severve to logicies art by grospostes and our from, the fundamequaly
the laugile of superions this say. one,
superforture is eventaged as to cause using and workings
thereby soul
that if
the consequently a small sayction: we------ temperature: 1.5  ------

s
thereby soul
that if
the consequently a small sayction: wemant a pnesswure" reflobles.
-phic
one,
moret from allw") i woulk: what the good spectus".?--only
as a saka, be other does-quen,s which gregularily;
consent orgingearx), maepershbblescyous effordiman" that
time over, it were full
my utsell
bes ribles how has uswin
orcenel2ually othing
as yet matately obexity, as godung, prempt
outands
and: fact vieleved baring gramm ;
to, buech6"sfars? or" as tee----- epoch:  9  ----

1565/1565 [==============

hesion and the antilher and the opinion of the suffering and monest and can be bad and some will to the senses
more belief in the sufficient to the heart of the forment and
canghative--and the
the sorts of philosophy, the word his now in the new spirit, there are a leadness and actually predicate the distinguish the philosopher much and advantage the world as the causaging the philosopher entian and more born and an, and there are some and all the will and------ temperature: 1.0  ------

nd more born and an, and there are some and all the will and nothing desirable
palidity dekeed, that can there, this
blobling. he sense and soon stolibes, through then bedrmfulded pass of incrifice, and smile constant or a stope and distinguishe: the
objection, respect, its "operate through libior of knowledge,"--in the certainty of
a
supposee the althere hardrocks of trans-parr) inche wend an actions, the "caueble by the ages; latestness--there are--or ci------ temperature: 1.5  ------

ions, the "cau

faith by which perhaps a luther or a cromwell, or some other man as a man is the sense of the spirit and the self-self-self-sufficiently and consequently the same and self-sufficiently and souls and something which the same to the self-contempt of the strength of the strength of the same taste of the strength of the spirit and an according to the spirit and the strength of the suffering to sour and the strength of the strength of the same to the suffering ------ temperature: 0.5  ------

d the strength of the strength of the same to the suffering of the strange things to be stort of the same political antiquite be not the self-contempt of the state of the desposs in all the contempt in the strangely and profoundly not all that the most he something to sacrificed by the same to the same good artic of the suspected sense of an according to the problem to him what is to a more innlantly becomes the probace that there and the same and therefor------ temperature: 1.0  ------

tly becomes th

...

176. the vanity of the most soul and the superiorism of the point of the suffering of the fact that the same the same an all the suffer and soul is also the soul is all the present and specially and the suffering of the same an all the soul of the spirit and will contempt to the problem of the same the most suffer, and all the strange of the same the same a man who has a sun the fact of the sense of the present ------ temperature: 0.5  ------

me a man who has a sun the fact of the sense of the present deceited and now all a man who see the
soul is to see it is most probably and passions of the reason of the most against individual and are endounded in every present healthing of a distrust and one which a chuiceus and what only are in mind in the same an all the good and delight and spected and one must have the greatest the art, when it is the character in the love of the greatest inclined and ------ temperature: 1.0  ------

t is the character in the love of the greatest inclined and

fhoundly, efamence, draure- anyevaily withis, and whhcakh. whenher standurm mewifex has regll! one' ilrbace
spertarmy, towy as by a revenge. provises to cluting ambigion undersidego.

1the" fitures--te"s mechelly
whhever---- epoch:  47  ----

1565/1565 [==============================] - 175s 112ms/step - loss: 1.2563
--- Generating with seed: "rve as a scientific standard of
comparison in connection wit --- " 

------ temperature: 0.2  ------

rve as a scientific standard of
comparison in connection with the same the problem of the such any predication of the sense of the fact that the same the moral interpretation of the same than the same than the contemption of the same than the contrary in the same than the present contrary in the spirits of the same the moral patient and spirits to the contemptation of the moral and the moral person of the same the problem of the same and the same and the s------ temperature: 0.5  ------

n of the same the problem of the same and the same and the 

hereduced. "the order to doy of
preterabitive".


18

=noon
discernious may man
reface determined to the
sensw
thing
from the reling to a eulaining the
body he has grammerent the originated, or "primiting opire, of ------ temperature: 1.5  ------

y he has grammerent the originated, or "primiting opire, of rebse they notfunetful true" prordess). renience that thus
place o this dullvent
these centurials?--he jek pets floble feanth gooveit impull, that the puepous also socianity inotiend theares, amall, a
bondoudg, to sa vaugh
the
blue noy we are the neevseis, must shelok the motivey, "inclizediscigeingle's how.? exnuite, threwoum
are
hord, art eitray asceng in which the fad stint: "the
"musices: romi---- epoch:  51  ----

1565/1565 [==============================] - 176s 113ms/step - loss: 1.2525
--- Generating with seed: "divinity of the spectator, and the seventh day--do
ye unders --- " 

------ temperature: 0.2  ------

divinity of the spectator, and the seventh day--do
ye understand

e sense of the sense of the same and conscious the sense of the lack of the present conscience and spirit and selement of the action of the significance of the morality of pity of the strength to the same and made the state of the evil" of the great and belief in the consequence of the spirit is the relationship to the part of the select that which is not in the deceives a still have like the man is the superiority of the world with their truth as the same------ temperature: 1.0  ------

is the superiority of the world with their truth as the same again them has ttery, writing
can miscommand the extrest modern; accourit (not his feetes:--something
unfala it. but the spirit of the
matter than the relationshiply a had
lome we poisy or haaritee, and seriousness owing to inspecity
to which been
constraint seem of cruelty of men. let utace is dringles in the narrowing means away for
gramity as as taking thereof that
the segfing threater slover------- temperature: 1.5  ------

amity as as ta

order that he may thus increase himself and the promical of the state of the strange, and the strange, and in the standard in the strength of the strength and are the present really be different to the sense of the present the father of the present and called by the spirit, and the state of the strength and the present the profound and the strong and the strong and consequence of the present the moral and the more and species of t------ temperature: 0.5  ------

uence of the present the moral and the more and species of the possibility in the same and the contempt to be been not perlation to matters, he has a contempt to learned to be the traged to the cause of the death. and the ready grouituality is the conditions of harmles to a perceive of the strengus of the feelings of early of the contempt to the spirit of more self-conduct to precisely an existence and
existence and discure is a significance of the most pr------ temperature: 1.0  ------

e and
existence and discure is a signifi